# Imports

In [14]:
from Bio import SeqIO
import csv
import math
import re

# Define lists

In [15]:
results = []
indexs = []
error_results =[]
error_indexs = []
sequence = []
extras = []
amino_acids = []
gene_names = []
accessions = []
functionalitys = []
partials = []

# Read in files

In [16]:
for seq_record in SeqIO.parse("tcrb_genomicJs.fasta", "fasta"):   
    ind = []
    extra_indexs = []
    three_amino_acids = []
    reading_frams = []
    # try out 3 different frames
    for i in range(3):
        
        # split record to match triplets
        seq_record_temp = seq_record.seq[i:]
        floor = math.floor(len(seq_record_temp)/3)
        index = len(seq_record_temp) - (floor*3)
        extra_indexs.append(index)
        if index != 0:
            seq_record_temp = seq_record_temp[:-(index)]
            
        # translating from dna to amino acid and find first (F/W)GXG(S/T)
        translated_seq = seq_record_temp.translate()
        three_amino_acids.append (translated_seq)
        position = -1
        m = re.search('[FW]G.?G[ST]', str(translated_seq))
        if m:
            position = m.start()
        index_F = ((position*3)+i)
        ind.append(index_F)
        reading_frams.append(i)
        
    #get the gene name
    if "|" not in seq_record.description: 
        gene_name = seq_record.description
        accession = ""
        functionality = ""
        partial = ""
    else:
        splitted = seq_record.description.split('|')
        gene_name = splitted[1]
        accession = splitted[0]
        functionality = splitted[3]
        partial = splitted[13]
    
    #look for only positive indexes
    pos_idx = [i for i in ind if i >=0]
    
    if len(pos_idx) != 0:
            pos_idx = min(pos_idx)
            
            #get the reading frame
            reading_frame_index = ind.index(pos_idx)
            reading_frame = reading_frame_index + 1
            
            #get the amino_acids with the correct reading frame
            amino_acid = three_amino_acids[reading_frame-1]
            
            # get the amino acids from the beginning to the first F 
            amino_acid_index = int((pos_idx-reading_frame+1)/3)+1
            amino_acid = amino_acid[0:amino_acid_index]
            
            pos_idx = str(pos_idx)
            
            indexs.append(pos_idx)
            results.append(seq_record.description)
            
            # get the extras 
            extra = seq_record.seq[0:reading_frame-1]
            
            extras.append(extra)
            amino_acids.append(amino_acid)
            gene_names.append(gene_name)
            accessions.append(accession)
            functionalitys.append(functionality)
            partials.append(partial)        
                   
    else:
            error_indexs.append(str(0))
            error_results.append(seq_record.description)
            sequence.append(seq_record.seq)
            



# write to file

In [17]:
with open('J_gene_CDR3_anchors_test.csv', 'w') as csv_file:
    fieldnames = ['gene','anchor_index']
    csv_writer = csv.DictWriter(csv_file,fieldnames=fieldnames,delimiter=';')
    csv_writer.writeheader()
    
    for result, index in zip(results, indexs):
        csv_writer.writerow({'gene': result, 'anchor_index': index})

# write to error file

In [18]:
with open('J_gene_CDR3_anchors_test_error', 'w') as csv_file:
        fieldnames = ['gene','sequence','anchor_index']
        csv_writer = csv.DictWriter(csv_file,fieldnames=fieldnames,delimiter=';')
        csv_writer.writeheader()

        for error_result, seq, error_index in zip(error_results, sequence, error_indexs):
            csv_writer.writerow({'gene': error_result, 'sequence':seq,'anchor_index': error_index})

# write to extra nucleotides file

In [19]:
with open('J_gene_CDR3_anchors_extra_nucleotides.csv', 'w') as csv_file:
    fieldnames = ['gene_name','extra_nucleotides','amino_acids','accession','functionality','partial']
    csv_writer = csv.DictWriter(csv_file,fieldnames=fieldnames,delimiter=';')
    csv_writer.writeheader()
    
    for gene_name, extra, amino_acid, accession, functionality, partial in zip(gene_names,extras,amino_acids,accessions,functionalitys,partials):
        csv_writer.writerow({'gene_name': gene_name,'extra_nucleotides': extra,'amino_acids':amino_acid,'accession':accession,'functionality':functionality,'partial':partial })